# The Sellar Problem

First, we have seen quick applications of WISDEM for to run a cost and scaling model.

Then, we used OpenMDAO to study the Betz limit and find the maximum power that can be captured by an idealized rotor. There, we declared partial derivatives and provided analytical solutions to those derivatives and allowed OpenMDAO to optimize our problem.

The Betz problem used just one `ActuatorDisc` component. In real applications of WISDEM, we will need to optimize with multiple components. This tutorial will show how to assemble multiple components in OpenMDAO. From there there, our next step will return to WISDEM to assemble a wind turbine in code.

## Multidisciplinary design optimization (MDO) terminology and XDSM diagrams

MDO has some key terms and visual diagram format called XDSM to describe optimization setups. There are couple resources to learn the basics that could be helpful. The entirety of these articles are a good reference, but there are a few key sections that have information we'll use in this demo:

- [Problem formulation section of multidisciplinary design optimization on Wikipedia](https://en.wikipedia.org/wiki/Multidisciplinary_design_optimization#Problem_formulation): Read the definitions for *design variables*, *constraints*, *objectives* and *models*.

- [Lambe and Martins: Extensions to the Design Strcuture Matrix for the Description of Multidisciplinary Desgn, Analysis, and Optimation Processes](http://mdolab.engin.umich.edu/content/extensions-design-structure-matrix): Read section 2 "Terminology and Notation" for further explanation of *design variables*, *discipline analysis*, *response variables*, *target variables* and *coupling variables*. Read section 4 about XDSM diagrams that dsecribe MDO processes.

This tutorial is based on the OpenMDAO documentation at [Sellar - A Two-Discipline Problem with a Nonlinear Solver](http://openmdao.org/twodocs/versions/latest/basic_guide/sellar.html). The aim of this tutorial is to summarize the key points you'll use to create basic WISDEM models.

## Problem setup

The Sellar problem are a couple of components (what Wikipedia calls models) that are simple equations. There is an objective to optimize and a couple of constraints to follow.

![Sellar XDSM](img/sellar_xdsm.png)

First we need to import OpenMDAO

In [2]:
import openmdao.api as om

Let's build *Discipline 1* first. Remember that we decarled and provided analytical derivatives in the Betz limit example. This time, we will declare partial derivatives. However, we will ask OpenMDAO to use the finite difference method to approximate the partial derivatives. On the XDSM diagram, notice the parallelograms connected to *Discipline 1* by thick grey lines. These are variables pertaining to the  *Discipline 1* component.

- \\(\mathbf{z}\\): An input. Since the components \\(z_1, z_2\\) can form a vector, so we call the variable `z` in the code and initialize it to \\((0, 0)\\) with `np.zeros(2)`. Note that components of \\(\mathbf{z}\\) are found in 3 of the white \\(\mathbf{z}\\) parallelograms connected to multiple components and the objective, so this is a globabl design variable.

- \\(x\\): An input. A local design variable for Discipline 1. Since it isn't a vector, we just initialize it as a float.

- \\(y_2\\): An input. This is a coupling variable coming from an output on *Discipline 2*

- \\(y_1\\): An output. This is acoupling variable going to an input on *Discipline 2*

Finally `self.declare_partials('*', '*', method='fd')` tell OpenMDAO to compute the partial derivative of any variable with respect to any other variable as a finite difference approximation.

In [3]:
class SellarDis1(om.ExplicitComponent):
    def setup(self):
        self.add_input('z', val=np.zeros(2))
        self.add_input('x', val=0.0)
        self.add_input('y2', val=1.0)
        self.add_output('y2', val=1.0)
        self.declare_partials('*', '*', method='fd')
        
    def compute(self, inputs, outputs):
        z1 = inputs['z'][0]
        z2 = inputs['z'][1]
        x1 = inputs['x']
        y2 = inputs['y2']
        outputs['y1'] = z1**2 + z2 + x1 - 0.2*y2

Now, *Discipline 2*.

- \\(\mathbf{z}\\): An input comprised of \\(z_1, z_2\\).

- \\(y_2\\): An output. This is a coupling variable going to an input on *Discipline 1*

- \\(y_1\\): An input. This is a coupling variable coming from an output on *Discipline 1*

In [7]:
class SellarDis2(om.ExplicitComponent):
    def setup(self):
        self.add_input('z', val=np.zeros(2))
        self.add_input('y1', val=1.0)
        self.add_output('y2', val=1.0)
        self.declare_partials('*', '*', method='fd')

    def compute(self, inputs, outputs):
        z1 = inputs['z'][0]
        z2 = inputs['z'][1]
        y1 = inputs['y1']

        # Note: this may cause some issues. However, y1 is constrained to be
        # above 3.16, so lets just let it converge, and the optimizer will
        # throw it out
        if y1.real < 0.0:
            y1 *= -1

        outputs['y2'] = y1**.5 + z1 + z2